In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
ops_per_sec = pd.read_csv('../data/ops_per_second.csv')

In [3]:
transistors_per_year = pd.read_csv('../data/transistor_count.csv')

In [4]:
life_exp = pd.read_csv('../data/life_expectancy.csv')

In [5]:
life_exp

,measure,location,sex,age,metric,year,val,upper,lower
0,Life expectancy,Somalia,Both,0-6 days,Years,1990,48.246310,50.913289,45.688151
1,Life expectancy,Somalia,Both,0-6 days,Years,1991,46.851486,49.400743,44.363649
2,Life expectancy,Somalia,Both,0-6 days,Years,1992,47.420054,49.981439,44.942891
3,Life expectancy,Somalia,Both,0-6 days,Years,1993,48.630956,51.325767,45.977770
4,Life expectancy,Somalia,Both,0-6 days,Years,1994,48.792347,51.504319,46.151635
...,...,...,...,...,...,...,...,...,...
6587,Life expectancy,High SDI,Both,0-6 days,Years,2017,80.898041,80.999959,80.790614
6588,Life expectancy,High SDI,Both,0-6 days,Years,2018,81.017264,81.122573,80.902723
6589,Life expectancy,High SDI,Both,0-6 days,Years,2019,81.174924,81.290505,81.044529
6590,Life expectancy,High SDI,Both,0-6 days,Years,2020,80.356340,80.475943,80.212811


In [6]:
life_exp = life_exp.drop(columns = {'upper', 'lower', 'measure', 'metric', 'age'}).rename(columns = {'val' : 'life_exp'})

In [7]:
death_counts = pd.read_csv('../data/death_counts.csv')

In [8]:
death_counts = death_counts.loc[death_counts['age'] == 'All ages']

In [9]:
death_counts = death_counts.drop(columns = {'measure', 'metric', 'upper', 'lower', 'age'}).rename(columns = {'val' : 'death_count'})

In [10]:
death_counts

,location,sex,cause,year,death_count
0,Myanmar,Both,Self-harm and interpersonal violence,1990,3921.694028
2,Germany,Both,Neglected tropical diseases and malaria,1990,32.256416
4,Bhutan,Both,Other non-communicable diseases,1990,176.498653
6,Bhutan,Both,Skin and subcutaneous diseases,1990,5.590641
8,Bhutan,Both,Transport injuries,1990,87.799638
...,...,...,...,...,...
263670,Zambia,Both,HIV/AIDS and sexually transmitted infections,2020,22742.154360
263672,Zambia,Both,Respiratory infections and tuberculosis,2020,22569.311186
263674,Zambia,Both,Enteric infections,2020,6070.105287
263676,Zambia,Both,Other infectious diseases,2020,3652.586152


In [11]:
population = pd.read_csv('../data/population.csv')

In [12]:
population = population.loc[population['age'] == 'All ages']

In [13]:
population = population.drop(columns = { 'measure', 'metric', 'upper', 'lower', 'age'}).rename(columns = {'val' : 'population'})

In [14]:
population

,location,sex,year,population
32,Iran (Islamic Republic of),Both,1990,5.710012e+07
33,Iran (Islamic Republic of),Both,1991,5.877901e+07
34,Iran (Islamic Republic of),Both,1992,6.014300e+07
35,Iran (Islamic Republic of),Both,1993,6.063039e+07
36,Iran (Islamic Republic of),Both,1994,6.063144e+07
...,...,...,...,...
13147,Bahrain,Both,2017,1.453405e+06
13148,Bahrain,Both,2018,1.473801e+06
13149,Bahrain,Both,2019,1.494478e+06
13150,Bahrain,Both,2020,1.513957e+06


In [15]:
death_rate = pd.read_csv('../data/death_rate(per100k).csv')

In [16]:
death_rate = death_rate.loc[death_rate['age'] == 'All ages']

In [17]:
death_rate = death_rate.drop(columns = {'measure', 'sex', 'age', 'metric', 'upper', 'lower'}).rename(columns = {'val' : 'death rate(per 100k)'})

In [18]:
death_rate

,location,cause,year,death rate(per 100k)
0,Myanmar,Mental disorders,1990,0.000051
1,Hungary,Nutritional deficiencies,1990,0.140524
3,Argentina,Cardiovascular diseases,1990,300.477206
5,South Africa,Neglected tropical diseases and malaria,1990,1.306245
7,Hungary,Digestive diseases,1990,88.364774
...,...,...,...,...
263670,Colombia,Nutritional deficiencies,2020,3.107113
263672,Palau,Transport injuries,2020,21.340618
263674,Palau,Unintentional injuries,2020,47.142279
263676,Colombia,Digestive diseases,2020,23.030370


In [19]:
prev_rate = pd.read_csv('../data/prevelance_rate(per100k).csv')

In [20]:
prev_rate = prev_rate.loc[prev_rate['age'] == 'All ages']

In [21]:
prev_rate = prev_rate.drop(columns = {'measure', 'sex', 'age', 'metric', 'upper', 'lower'})

In [22]:
prev_rate = prev_rate.rename(columns = {'val' : 'prevelance rate(per 100k)'})

In [23]:
death_data = death_counts.merge(death_rate, how = 'left').merge(prev_rate, how = 'left').merge(population, how = 'left').merge(life_exp, how = 'left')

In [24]:
death_data

,location,sex,cause,year,death_count,death rate(per 100k),prevelance rate(per 100k),population,life_exp
0,Myanmar,Both,Self-harm and interpersonal violence,1990,3921.694028,9.697932,4212.974669,4.043846e+07,55.056371
1,Germany,Both,Neglected tropical diseases and malaria,1990,32.256416,0.040350,405.302074,7.994078e+07,75.542568
2,Bhutan,Both,Other non-communicable diseases,1990,176.498653,28.013198,64623.131137,6.300554e+05,60.161661
3,Bhutan,Both,Skin and subcutaneous diseases,1990,5.590641,0.887325,20369.993241,6.300554e+05,60.161661
4,Bhutan,Both,Transport injuries,1990,87.799638,13.935226,617.861068,6.300554e+05,60.161661
...,...,...,...,...,...,...,...,...,...
138427,Zambia,Both,HIV/AIDS and sexually transmitted infections,2020,22742.154360,119.521340,31422.924483,1.902769e+07,60.928404
138428,Zambia,Both,Respiratory infections and tuberculosis,2020,22569.311186,118.612963,23781.867915,1.902769e+07,60.928404
138429,Zambia,Both,Enteric infections,2020,6070.105287,31.901424,1526.326568,1.902769e+07,60.928404
138430,Zambia,Both,Other infectious diseases,2020,3652.586152,19.196158,3185.055190,1.902769e+07,60.928404


In [25]:
death_data['lethality'] = (death_data['death rate(per 100k)'] / death_data['prevelance rate(per 100k)']) * 100

In [26]:
death_data

,location,sex,cause,year,death_count,death rate(per 100k),prevelance rate(per 100k),population,life_exp,lethality
0,Myanmar,Both,Self-harm and interpersonal violence,1990,3921.694028,9.697932,4212.974669,4.043846e+07,55.056371,0.230192
1,Germany,Both,Neglected tropical diseases and malaria,1990,32.256416,0.040350,405.302074,7.994078e+07,75.542568,0.009956
2,Bhutan,Both,Other non-communicable diseases,1990,176.498653,28.013198,64623.131137,6.300554e+05,60.161661,0.043349
3,Bhutan,Both,Skin and subcutaneous diseases,1990,5.590641,0.887325,20369.993241,6.300554e+05,60.161661,0.004356
4,Bhutan,Both,Transport injuries,1990,87.799638,13.935226,617.861068,6.300554e+05,60.161661,2.255398
...,...,...,...,...,...,...,...,...,...,...
138427,Zambia,Both,HIV/AIDS and sexually transmitted infections,2020,22742.154360,119.521340,31422.924483,1.902769e+07,60.928404,0.380364
138428,Zambia,Both,Respiratory infections and tuberculosis,2020,22569.311186,118.612963,23781.867915,1.902769e+07,60.928404,0.498754
138429,Zambia,Both,Enteric infections,2020,6070.105287,31.901424,1526.326568,1.902769e+07,60.928404,2.090079
138430,Zambia,Both,Other infectious diseases,2020,3652.586152,19.196158,3185.055190,1.902769e+07,60.928404,0.602695


In [27]:
pc_performance = transistors_per_year.merge(ops_per_sec, how = 'left')

In [28]:
pc_performance = pc_performance.rename(columns = {'Transistors per microprocessor' : 'transistor_count(thousand)', 'Floating-Point Operations per Second (GFLOPS)' : 'GFLOPS'})

In [29]:
pc_performance['transistor_count(thousand)'] = (pc_performance['transistor_count(thousand)'] / 1000).astype(int)

In [30]:
pc_performance = pc_performance.drop(columns = {'Entity', 'Code'})

In [31]:
zero_death_count = death_data.loc[death_data['death_count'] == 0]

In [32]:
death_data = death_data[~death_data.isin(zero_death_count)]

In [33]:
death_data

,location,sex,cause,year,death_count,death rate(per 100k),prevelance rate(per 100k),population,life_exp,lethality
0,Myanmar,Both,Self-harm and interpersonal violence,1990,3921.694028,9.697932,4212.974669,4.043846e+07,55.056371,0.230192
1,Germany,Both,Neglected tropical diseases and malaria,1990,32.256416,0.040350,405.302074,7.994078e+07,75.542568,0.009956
2,Bhutan,Both,Other non-communicable diseases,1990,176.498653,28.013198,64623.131137,6.300554e+05,60.161661,0.043349
3,Bhutan,Both,Skin and subcutaneous diseases,1990,5.590641,0.887325,20369.993241,6.300554e+05,60.161661,0.004356
4,Bhutan,Both,Transport injuries,1990,87.799638,13.935226,617.861068,6.300554e+05,60.161661,2.255398
...,...,...,...,...,...,...,...,...,...,...
138427,Zambia,Both,HIV/AIDS and sexually transmitted infections,2020,22742.154360,119.521340,31422.924483,1.902769e+07,60.928404,0.380364
138428,Zambia,Both,Respiratory infections and tuberculosis,2020,22569.311186,118.612963,23781.867915,1.902769e+07,60.928404,0.498754
138429,Zambia,Both,Enteric infections,2020,6070.105287,31.901424,1526.326568,1.902769e+07,60.928404,2.090079
138430,Zambia,Both,Other infectious diseases,2020,3652.586152,19.196158,3185.055190,1.902769e+07,60.928404,0.602695


In [34]:
cost_per_gflop = pd.read_csv('../data/cost_of_computing.csv', index_col = 0)

In [35]:
cost_per_gflop = cost_per_gflop.rename(columns = {'Unadjusted' : 'unadjusted_cost_per_gflop', '2023[77]' : 'inflation_adjusted_cost_per_gflop', 'year' : 'Year'})

In [36]:
pc_performance = cost_per_gflop.merge(pc_performance, how = 'left')

In [37]:
storage_costs = pd.read_csv('../data/storage_costs.csv')

In [38]:
storage_costs = storage_costs.drop(columns = {'Entity', 'Code'})

In [39]:
storage_costs['Historical price of memory'] = storage_costs['Historical price of memory'] / 1000

In [40]:
storage_costs = storage_costs.rename(columns = {'Historical price of memory' : 'price_of_memory(GB)', 'Historical price of flash memory' : 'price_of_flash_memory(TB)', 'Historical price of disk drives' : 'price_of_disk_storage(TB)', 'Historical price of solid-state drives' : 'price_of_solid_state_storage(TB)'})

In [41]:
pc_performance = pc_performance.merge(storage_costs, how = 'right')

In [42]:
pc_performance

,Year,unadjusted_cost_per_gflop,inflation_adjusted_cost_per_gflop,transistor_count(thousand),GFLOPS,price_of_memory(GB),price_of_flash_memory(TB),price_of_disk_storage(TB),price_of_solid_state_storage(TB)
0,1957,NaN,NaN,NaN,NaN,3.786967e+12,NaN,NaN,NaN
1,1959,NaN,NaN,NaN,NaN,6.032806e+11,NaN,6.747736e+10,NaN
2,1960,NaN,NaN,NaN,NaN,4.588051e+10,NaN,3.150364e+10,NaN
3,1965,NaN,NaN,NaN,NaN,2.170492e+10,NaN,NaN,NaN
4,1970,NaN,NaN,NaN,NaN,4.892936e+09,NaN,1.731185e+09,NaN
...,...,...,...,...,...,...,...,...,...
60,2022,$0.02,$0.02,NaN,NaN,1.861932e+00,NaN,1.263745e+01,39.796032
61,2023,$0.01,$0.01,NaN,NaN,1.088437e+00,NaN,1.061740e+01,25.906467
62,1956,NaN,NaN,NaN,NaN,NaN,NaN,8.759275e+10,NaN
63,1964,NaN,NaN,NaN,NaN,NaN,NaN,2.936028e+10,NaN


In [43]:
death_data_countries = death_data.loc[(death_data['location'] != 'High SDI') & (death_data['location'] != 'Low SDI')]

In [44]:
death_data_low = death_data.loc[(death_data['location'] == 'Low SDI')]

In [50]:
death_data_high = death_data.loc[(death_data['location'] == 'High SDI')]

In [52]:
death_data_economic = death_data.loc[(death_data['location'] == 'High SDI') | (death_data['location'] == 'Low SDI')]

In [54]:
death_data_economic

,location,sex,cause,year,death_count,death rate(per 100k),prevelance rate(per 100k),population,life_exp,lethality
1262,Low SDI,Both,Skin and subcutaneous diseases,1991,6.682926e+03,1.301913,27140.967132,5.133160e+08,53.197732,4.796854e-03
1263,Low SDI,Both,Transport injuries,1991,1.297379e+05,25.274479,964.485797,5.133160e+08,53.197732,2.620513e+00
1312,Low SDI,Both,HIV/AIDS and sexually transmitted infections,1991,2.468401e+05,48.087368,17361.438551,5.133160e+08,53.197732,2.769780e-01
1313,Low SDI,Both,Respiratory infections and tuberculosis,1991,1.242490e+06,242.051699,38166.964052,5.133160e+08,53.197732,6.341916e-01
1314,Low SDI,Both,Enteric infections,1991,9.890918e+05,192.686738,2446.780207,5.133160e+08,53.197732,7.875114e+00
...,...,...,...,...,...,...,...,...,...,...
137004,High SDI,Both,Substance use disorders,2020,1.263413e+05,11.579874,3807.984263,1.091042e+09,80.356340,3.040946e-01
137005,High SDI,Both,Diabetes and kidney diseases,2020,5.391457e+05,49.415654,18766.519964,1.091042e+09,80.356340,2.633182e-01
137184,Low SDI,Both,Nutritional deficiencies,2020,8.407520e+04,7.696935,44871.699439,1.092320e+09,63.528116,1.715321e-02
137311,Low SDI,Both,Neurological disorders,2020,1.147844e+05,10.508305,29016.413094,1.092320e+09,63.528116,3.621504e-02


In [56]:
death_data_countries

,location,sex,cause,year,death_count,death rate(per 100k),prevelance rate(per 100k),population,life_exp,lethality
0,Myanmar,Both,Self-harm and interpersonal violence,1990,3921.694028,9.697932,4212.974669,4.043846e+07,55.056371,0.230192
1,Germany,Both,Neglected tropical diseases and malaria,1990,32.256416,0.040350,405.302074,7.994078e+07,75.542568,0.009956
2,Bhutan,Both,Other non-communicable diseases,1990,176.498653,28.013198,64623.131137,6.300554e+05,60.161661,0.043349
3,Bhutan,Both,Skin and subcutaneous diseases,1990,5.590641,0.887325,20369.993241,6.300554e+05,60.161661,0.004356
4,Bhutan,Both,Transport injuries,1990,87.799638,13.935226,617.861068,6.300554e+05,60.161661,2.255398
...,...,...,...,...,...,...,...,...,...,...
138427,Zambia,Both,HIV/AIDS and sexually transmitted infections,2020,22742.154360,119.521340,31422.924483,1.902769e+07,60.928404,0.380364
138428,Zambia,Both,Respiratory infections and tuberculosis,2020,22569.311186,118.612963,23781.867915,1.902769e+07,60.928404,0.498754
138429,Zambia,Both,Enteric infections,2020,6070.105287,31.901424,1526.326568,1.902769e+07,60.928404,2.090079
138430,Zambia,Both,Other infectious diseases,2020,3652.586152,19.196158,3185.055190,1.902769e+07,60.928404,0.602695


In [58]:
lat_long = pd.read_csv('../data/longitude-latitude.csv')

In [60]:
lat_long = lat_long.rename(columns = {'Country' : 'location'})

In [62]:
death_data_countries = death_data_countries.merge(lat_long, how = 'left')

In [64]:
death_data_countries.to_csv('../cleaned_data/death_data_countries.csv', index = False)

In [65]:
death_data_low.to_csv('../cleaned_data/death_data_low.csv', index = False)

In [66]:
death_data_high.to_csv('../cleaned_data/death_data_high.csv', index = False)

In [72]:
death_data_economic.to_csv('../cleaned_data/death_data_economic')

In [74]:
pc_performance.to_csv('../cleaned_data/pc_performance.csv', index = False)